# Clustering with KMeans

## Task Description

Create a dummy dataset with 3 different labels: [a, b, c] and cluster them using KMeans. Then classify the dummy dataset using kmeans model and print out the rows of the first cluster. Finally save the kmeans model to kmeans.mlnet. Once completed, print 'task resolved' in the end.

## Task Description

Create a dummy dataset with 3 different labels: [a, b, c] and cluster them using KMeans. Then classify the dummy dataset using kmeans model and print out the rows of the first cluster. Finally save the kmeans model to kmeans.mlnet. Once completed, print 'task resolved' in the end.

In [ ]:
#r "nuget:Microsoft.ML"


In [ ]:
using System;
using System.Collections.Generic;
using System.Linq;
using Microsoft.ML;
using Microsoft.ML.Data;

var mlContext = new MLContext(seed: 0);

// Create a list of training data points.
var dataPoints = GenerateRandomDataPoints(1000, 123);

// Convert the list of data points to an IDataView object, which is
// consumable by ML.NET API.
IDataView trainingData = mlContext.Data.LoadFromEnumerable(dataPoints);

// Define the trainer.
var pipeline = mlContext.Clustering.Trainers.KMeans(
    numberOfClusters: 3);

// Train the model.
var model = pipeline.Fit(trainingData);

// Create testing data. Use a different random seed to make it different
// from the training data.
var testData = mlContext.Data.LoadFromEnumerable(
    GenerateRandomDataPoints(500, seed: 123));

// Run the model on test data set.
var transformedTestData = model.Transform(testData);

// Convert IDataView object to a list.
var predictions = mlContext.Data.CreateEnumerable<Prediction>(
    transformedTestData, reuseRowObject: false).ToList();

// Print 5 predictions. Note that the label is only used as a comparison
// with the predicted label. It is not used during training.
foreach (var p in predictions.Where(x => x.PredictedLabel == 1))
    Console.WriteLine(
        $"Label: {p.Label}, Prediction: {p.PredictedLabel}");

// Save the model
mlContext.Model.Save(model, trainingData.Schema, "kmeans.mlnet");

Console.WriteLine("task resolved");

private static IEnumerable<DataPoint> GenerateRandomDataPoints(int count,
    int seed = 0)
{
    var random = new Random(seed);
    float randomFloat() => (float)random.NextDouble();
    for (int i = 0; i < count; i++)
    {
        int label = i < count / 3 ? 0 : (i < 2 * count / 3 ? 1 : 2);
        yield return new DataPoint
        {
            Label = (uint)label,
            // Create random features with three clusters.
            // The first third has feature values centered around 0.6, 
            // the second third has values centered around 0.4,
            // and the last third has values centered around 0.2.
            Features = Enumerable.Repeat(label, 50)
                .Select(index => label == 0 ? randomFloat() + 0.1f :
                    (label == 1 ? randomFloat() : randomFloat() - 0.1f)).ToArray()
        };
    }
}

// Example with label and 50 feature values. A data set is a collection of
// such examples.
private class DataPoint
{
    // The label is not used during training, just for comparison with the
    // predicted label.
    [KeyType(3)]
    public uint Label { get; set; }

    [VectorType(50)]
    public float[] Features { get; set; }
}

// Class used to capture predictions.
private class Prediction
{
    // Original label (not used during training, just for comparison).
    public uint Label { get; set; }
    // Predicted label from the trainer.
    public uint PredictedLabel { get; set; }
}